# KcELECTRA-base
 : 왜인진 모르겠는데 beomi/KcELECTRA-base는 글자가 깨져서 beomi/KcELECTRA-base-v2022로 실행함

## KcELECTRA
- Korean comments Efficiently Learning an Encoder that Classifies Token Replacements Accurately
- 학습 방식
    - 입력된 토큰을 Generator에 의해 그럴듯한 가짜 토큰으로 대체한다.
    - 이후 토큰이 가짜 토큰인지 실제 토큰인지 이진분류 방식으로 학습한다.
    - 이러한 이진분류는 모든 토큰에 진행되므로 기존 MLM 방식보다 훨씬 효율적이다.
- Tokenizer
	- BertWordPieceTokenizer

[KcELECTRA 악성댓글 분류 모델 만들기](https://www.dinolabs.ai/400)

## 환경설정

`pip install pandas scikit-learn`

[data 출처: github ZIZUN/korean-malicious-comments-dataset](https://github.com/ZIZUN/korean-malicious-comments-dataset/blob/master/Dataset.csv)

In [1]:
from src.Common.common import *

"""
데이터 조작 및 분석을 위한 라이브러리
"""
import pandas as pd

"""
HuggingFace 라이브러리
- AutoTokenizer: huggingface 모델용 tokenizer 추상 객체
- AutoModelForSequenceClassification: huggingface 모델 추상 객체, 입력 문장 label에 맞게 분류하는 모델
- TrainingArguments: 모델 학습 하이퍼 파라미터 객체
- Trainer: 모델 학습용 객체
"""
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

c = CommonObj()

c.KcELECTRA_MODEL_NAME = "beomi/KcELECTRA-base-v2022"
c.SAVED_MODEL_NAME = "../../models/KcELECTRA"

## Dataset 제작

In [2]:
"""
\t 문자로 분리된 csv 파일 읽어오기
"""
c.df = pd.read_csv("../../datas/KcELECTRA/Dataset.csv", sep = "\t")

"""
읽어온 csv 파일의 앞 5개 row 출력
"""
c.df.head()

,content,lable
0,이종석 한효주 나오는 드라마 이후로 드라마 안봤다. 2년전인가?? 좀 신선했었지. ...,0.0
1,씨바알..노무노무 술프노... 오늘 저녁은 꽂등심이다ㅠㅜ,0.0
2,짱깨 꺼라ㅡ패쓰,0.0
3,그들의 사생활 ~ 고인이된 설리를 위해서라도 모두 조용하길 지금 누굴 탓한다고 무슨...,1.0
4,아무리 법이 뭣같아도 무슨 자격으로 개인의 신상정보를 불특정 다수에게 공개하는지 도...,1.0


## null label 제거

In [3]:
c.null_idx = c.df[c.df.lable.isnull()].index
c.df.loc[c.null_idx, "content"]

1602    응애 응애 엄마 저 맘에 안들죠? ........아들 ?? " 너 내가 우스워 보이...
1654           토니스타크 평소 "아이엠그루트"라는 유행어를 부러워했다는게 학계의 정설\t1
1992    "13일 현대차에 따르면 올 들어 국내 소비자들의 수입차 구매의향률이 3년 만에 하...
2920                 에이프릴이 한마디 합니다 "예쁜게 죄" 구하라님 "무기징역"\t1
3720          답글 글씨체를 봐라 저게 애새끼가 쓴거냐?"빨갱이새끼가 쓴거지 ㅁㅈㅎㅉㅉ\t0
3807    알겠다이기ㅋㅋ 딱 채찍쳐맞는거 좋아하는 한국식 마인드네. 노예마인드. 조금만 성공한...
3908           이래서 스스로 걸리거든 "죄인들이"~ㅎㅎㅎ 재미보고 털리고 그치~~~?\t0
4241    아버지는 내재된 악마들을 다룰 정신적 힘을 가지고 있지 않았다." 이 말한마디가 사...
4283    댓글 중 "선동 당해서 촞불든 개돼지 홍어들도 단죄를 받아야 할 공범자들이다"에10...
5000    스파이 제안받고 살해 안당하는 법1. 처음에 스파이 제안을 받았을때 "중국을 위해서...
5521    "국방부 "까지 ㅡㄱ ㅐ 엿같은 ㅈ ㅣ랄주댕이...좌빨에서 ㅡ인민군대로 ㅡ가려는건가...
5866    쌩뚱맞게 60대최반엌 치매라니 그것도 곱게 사는 사모님이- -" 알콜중독도 아니고 ...
6477    페미메퇘지쿵쾅년인 메갈페미들은 니들이 좋아하는 싫어요 ㄱㄱ제발부탁해~~"일반 여성"...
6538    아니 ㅆㅂ 그런 "카더라"가 넘쳐난다고 그거에 대해서 혹시 댓글게이는 뭔가 아는거 ...
6771    저 때 투니버스에서 코요태 짧게 인터뷰 했었는데 김종민이 "노래는 뭐 신지가 다 하...
6932               개 족 가튼 국방부의 "휴기연장콜센터"발족을 축하한다 ㅆ ㅂ..\t0
7199    민족적 자존심과 애국심을 갖고 국산품 이용합시다 . . . "겸손"한 마음으로 재산...
7252    아나운서는 

In [4]:
# lable은 content의 가장 끝 문자열
c.df.loc[c.null_idx, "lable"] = c.df.loc[c.null_idx, "content"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열
c.df.loc[c.null_idx, "content"] = c.df.loc[c.null_idx, "content"].apply(lambda x: x[:-2])


In [5]:
c.df = c.df.astype({"lable": "int64"})
c.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  10000 non-null  object
 1   lable    10000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 156.4+ KB


## Train set / Test set

train_set : test_set = 80 : 20

In [6]:
c.train_data = c.df.sample(frac = 0.8, random_state = 42)
c.test_data = c.df.drop(c.train_data.index)

# dataset 개수 확인
print(f"중복 제거 전 train_data 개수: {len(c.train_data)}")
print(f"중복 제거 전 test_data 개수: {len(c.test_data)}")

# 중복 데이터 제거
c.train_data.drop_duplicates(subset = ["content"], inplace = True)
c.test_data.drop_duplicates(subset = ["content"], inplace = True)

# dataset 개수 확인
print(f"중복 제거 후 train_data 개수: {len(c.train_data)}")
print(f"중복 제거 후 test_data 개수: {len(c.test_data)}")

중복 제거 전 train_data 개수: 8000
중복 제거 전 test_data 개수: 2000
중복 제거 후 train_data 개수: 7992
중복 제거 후 test_data 개수: 2000


## Tokenizer

In [7]:
# c.tokenizer = AutoTokenizer.from_pretrained(c.KcELECTRA_MODEL_NAME)
c.tokenizer = AutoTokenizer.from_pretrained(c.SAVED_MODEL_NAME)

print(c.tokenizer)

BertTokenizerFast(name_or_path='../../models/KcELECTRA', vocab_size=54343, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)


In [8]:
c.tokenized_train_sentences = c.tokenizer(
	list(c.train_data["content"]),
	return_tensors = "pt",
	max_length = 128,
	padding = True,
	truncation = True,
	add_special_tokens = True
)

c.tokenized_test_sentences = c.tokenizer(
	list(c.test_data["content"]),
	return_tensors = "pt",
	max_length = 128,
	padding = True,
	truncation = True,
	add_special_tokens = True
)

In [9]:
print(c.tokenized_train_sentences[0])
print(c.tokenized_train_sentences[0].tokens)
print(c.tokenized_train_sentences[0].ids)
print(c.tokenized_train_sentences[0].attention_mask)

Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '국방부', '~', '~', '전화로', '휴가', '##연장', '##을', '한', '병사', '##들', '몇이나', '되는지', '공개해라', '~', '어느', '훌륭한', '집안', '##의', '자제', '##분들', '##인지도', '같이', '공개해라', '~', '~', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

## Dataset 생성

In [10]:
from torch.utils.data import Dataset
class CurseDataset(torch.utils.data.Dataset):
	def __init__(self, encodings, labels):
		self.encodings = encodings
		self.labels = labels
	
	def __getitem__(self, idx):
		item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
		item["labels"] = torch.tensor(self.labels[idx])
		return item
	
	def __len__(self):
		return len(self.labels)

c.train_dataset = CurseDataset(c.tokenized_train_sentences, c.train_data["lable"].values)
c.test_dataset = CurseDataset(c.tokenized_test_sentences, c.test_data["lable"].values)

## 모델 Load

In [11]:
# c.model = AutoModelForSequenceClassification.from_pretrained(c.KcELECTRA_MODEL_NAME, num_labels = 2).to("cuda:0")
c.model = AutoModelForSequenceClassification.from_pretrained(c.SAVED_MODEL_NAME, num_labels = 2).to("cuda:0")

## Training Arguments

In [12]:
c.training_args = TrainingArguments(
	output_dir = "./",
	num_train_epochs = 10,
	per_device_train_batch_size = 8,
	per_device_eval_batch_size = 64,
	logging_dir = "./logs",
	logging_steps = 500, # 학습 log 기록 단위
	log_level = "info",
	save_total_limit = 2 # 학습 결과 저장 최대 갯수
)

## Trainer

In [13]:
c.trainer = Trainer(
	model = c.model,
	args = c.training_args,
	train_dataset = c.train_dataset,
	eval_dataset = c.test_dataset,
	compute_metrics = compute_metrics
)

## Training

In [14]:
c.trainer.train()

c.model.save_pretrained(c.SAVED_MODEL_NAME)
c.tokenizer.save_pretrained(c.SAVED_MODEL_NAME)

C:\Users\jongg\PycharmProjects\NLP\venv\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7,992
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9,990
  Number of trainable parameters = 127,778,306


Step,Training Loss


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500\config.json
Model weights saved in ./checkpoint-500\pytorch_model.bin
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000\config.json
Model weights saved in ./checkpoint-1000\pytorch_model.bin
Saving model checkpoint to ./checkpoint-1500
Configuration saved in ./checkpoint-1500\config.json
Model weights saved in ./checkpoint-1500\pytorch_model.bin
Deleting older checkpoint [checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./checkpoint-2000
Configuration saved in ./checkpoint-2000\config.json
Model weights saved in ./checkpoint-2000\pytorch_model.bin
Deleting older checkpoint [checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to ./checkpoint-2500
Configuration saved in ./checkpoint-2500\config.json
Model weights saved in ./checkpoint-2500\pytorch_model.bin
Deleting older checkpoint [checkpoint-1500] due to args.save_total_limit


('../../models/KcELECTRA\\tokenizer_config.json',
 '../../models/KcELECTRA\\special_tokens_map.json',
 '../../models/KcELECTRA\\vocab.txt',
 '../../models/KcELECTRA\\added_tokens.json',
 '../../models/KcELECTRA\\tokenizer.json')

## Model Evaluate

In [15]:
c.trainer.evaluate(eval_dataset = c.test_dataset)

***** Running Evaluation *****
  Num examples = 2000
  Batch size = 64


{'eval_loss': 1.0783494710922241,
 'eval_accuracy': 0.8935,
 'eval_f1': 0.8949185989146522,
 'eval_precision': 0.8712776176753122,
 'eval_recall': 0.9198782961460447,
 'eval_runtime': 7.7176,
 'eval_samples_per_second': 259.149,
 'eval_steps_per_second': 4.146,
 'epoch': 10.0}

## 모델 적용

In [16]:
# 0: 악성댓글 1: 정상댓글
def sentence_predict(sent):
	# 평가모드
	c.model.eval()
	
	# 입력된 문장 tokenizing
	tokenized_sent = c.tokenizer(sent,
							   return_tensors = "pt",
							   truncation = True,
							   add_special_tokens = True,
							   max_length = 128)
	
	tokenized_sent.to("cuda:0")
	
	# 예측
	with torch.no_grad():
		outputs = c.model(**tokenized_sent)
	
	# 결과 return
	logits = outputs[0]
	logits = logits.detach().cpu()
	result = logits.argmax(-1)
	
	return "정상댓글" if result == 1 else "악성댓글"

In [17]:
sentence = input("댓글입력: ")
print(f"입력한 댓글 \"{sentence}\"는 {sentence_predict(sentence)}입니다.")

입력한 댓글 "염병할 놈들"는 악성댓글입니다.


In [18]:
del c


/*====================================================================*/
device:0
NVIDIA GeForce RTX 3060

GPU memory occupied: 9059 MB.

Allocated GPU Memory: 1.46GB
Reserved GPU Memory: 1.63GB
/*====================================================================*/
